# PART II
The purpose of this document is to split the forcing data and the velocity results for a single member. First, we load in the relevant libraries and packages.

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob
import intake

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34241")
client

<Client: 'tcp://127.0.0.1:34241' processes=4 threads=8, memory=29.39 GiB>

In [6]:
cat = intake.open_catalog('https://raw.githubusercontent.com/ldeo-glaciology/pangeo-pismpaleo/main/paleopism.yaml')
snapshots = cat["snapshots1ka"].to_dask()

In [16]:
vels = snapshots.velbar_mag
vels_train = vels.isel(par_esia=[0,2,3],par_prec = [0,1,3],par_visc=[0,1,3],par_ppq=[0,1,3])
vels_train

,Array,Chunk
Bytes,5.48 GiB,373.78 MiB
Shape,"(125, 381, 381, 3, 3, 3, 3)","(25, 381, 381, 1, 3, 3, 3)"
Count,6 Graph Layers,15 Chunks
Type,float32,numpy.ndarray


In [28]:
ds_forcing = xr.open_dataset('timeseries_edc-wdc_temp.nc',decode_times=False)
ds_forcing_interp = ds_forcing.interp(time=snapshots.time)
ds_forcing_interp

<xarray.Dataset>
Dimensions:  (time: 125)
Coordinates:
  * time     (time) float64 -1.24e+05 -1.23e+05 -1.22e+05 ... -2e+03 -1e+03 0.0
Data variables:
    delta_T  (time) float64 1.625 2.092 1.347 1.88 ... 0.1106 0.24 0.03 0.0
Attributes:
    NCO:                       netCDF Operators version 4.7.8 (Homepage = htt...
    nco_openmp_thread_number:  1
    source_EDC:                ftp://ftp.ncdc.noaa.gov/pub/data/paleo/icecore...
    source_WDC:                http://www.usap-dc.org/view/dataset/600377
    processing_EDC:            https://github.com/pism/pism-ais/tree/master/edc
    processing_WDC:            https://github.com/pism/pism-ais/tree/master/wdc
    history:                   Thu Oct 13 17:59:53 2022: ncatted -O -a histor...

In [106]:
Y_all = []
length_all = []
X_all      = []

for i in [0,1,2]:
    for j in [0,1,2]:
        for k in [0,1,2]:
            for l in [0,1,2]:
                data = vels_train.isel(par_esia=i,par_prec=j,par_visc=k,par_ppq=l)
                Y_all.append(data)
                length_all.append(len(data.time))
                X_all.append(ds_forcing_interp.delta_T)

length_all = np.array(length_all)
Y = xr.concat(Y_all,dim='time')
# Y = xr.concat([xr.open_dataset(data_path + f"outputs_{file}.nc") for file in data_sets], dim='time').mean("member")
Y = Y.transpose('time','x', 'y')
Y = Y.assign_coords(time=np.arange(len(Y.time)))


X = xr.concat(X_all,dim='time')
X = X.transpose('time')
X = X.assign_coords(time=np.arange(len(X.time)))
X = X.to_dataset()
X['par_esia'] = (['time'],Y.par_esia.values)
X['par_ppq'] = (['time'],Y.par_ppq.values)
X['par_prec'] = (['time'],Y.par_prec.values)
X['par_visc'] = (['time'],Y.par_visc.values)

X_train_xr = X
y_train_xr = Y.drop(['par_esia','par_ppq','par_prec','par_visc'])

In [107]:
vels = xr.open_dataset('raw_data/velsurf_165_100yr_d1.nc',decode_times=False)
vels['time'] = vels.time/31536000
y_test_xr = vels.velsurf_mag
X_test_xr = ds_forcing.interp(time=Y_test_xr.time)
X_test_xr['par_esia'] = (['time'],2.0*np.ones(len(Y_test_xr.time)))
X_test_xr['par_ppq'] = (['time'],0.75*np.ones(len(Y_test_xr.time)))
X_test_xr['par_prec'] = (['time'],0.07*np.ones(len(Y_test_xr.time)))
X_test_xr['par_visc'] = (['time'],5e20*np.ones(len(Y_test_xr.time)))

In [108]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}
predictors     = ['delta_T','par_esia','par_ppq','par_prec','par_visc']
for var in predictors:
    meanstd_inputs[var] = (X_train_xr[var].data.mean(),X_train_xr[var].data.std())
    
# normalize each variables
for var in predictors:
    # training set
    var_dims   = X_train_xr[var].dims
    X_train_xr = X_train_xr.assign({var: (var_dims, normalize(X_train_xr[var].data, var, meanstd_inputs))})
    
    # test set
    var_dims  = X_test_xr[var].dims
    X_test_xr = X_test_xr.assign({var: (var_dims, normalize(X_test_xr[var].data, var, meanstd_inputs))})

In [116]:
y_train = y_train_xr.fillna(-1).data
y_test  = y_test_xr.fillna(-1).data

In [117]:
X_train = X_train_xr.to_array().transpose('time', 'variable').data
X_test  = X_test_xr.to_array().transpose('time', 'variable').data

print(X_train_np.shape,y_train_np.shape,X_test_np.shape,y_test_np.shape)

(10125, 5) (10125, 381, 381) (1249, 5) (1249, 381, 381)


In [119]:
start = np.cumsum(length_all) - length_all
end   = np.cumsum(length_all)

slider = 2
X_train_all = []
y_train_all = []

for i in range(len(length_all)):
    
    X_subset = X_train[start[i]:end[i],:]
    y_subset = y_train[start[i]:end[i],:]
    
    X_subset = np.array([X_subset[i:i+slider] for i in range(0, length_all[i]-slider+1)])
    y_subset = np.array([[y_subset[i+slider-1]] for i in range(0, length_all[i]-slider+1)])
    
    X_train_all.append(X_subset)
    y_train_all.append(y_subset)
    
X_train = np.concatenate(X_train_all,axis=0)
y_train = np.concatenate(y_train_all,axis=0)
X_test  = np.array([X_test[i:i+slider] for i in range(0, X_test.shape[0]-slider+1)])
print(X_train.shape,y_train.shape,X_test.shape)

KeyboardInterrupt: 